<a href="https://colab.research.google.com/github/deviljerry/Decoder-Only-GPT-2-or-LLaMA-Pseudo-code-to-Code-Generation.ipynb/blob/main/Decoder_Only_(GPT_2_or_LLaMA)_%E2%80%94Pseudo_code_to_Code_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================================
# 🚀 TASK 2: Decoder-Only (GPT-2) — Pseudo-code → Python
# ======================================================

# 1️⃣ Install required libraries
!pip install transformers datasets evaluate torch sentencepiece gradio sacrebleu -q

# 2️⃣ Import libraries
import os, pandas as pd, torch
from datasets import Dataset
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling
)
import evaluate, gradio as gr
from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.0 MB/s eta 0:00:00
✅ Device: cuda


In [2]:
# ======================================================
# ⚙️ Step 1: Download SPOC Dataset from GitHub
# ======================================================
!git clone https://github.com/sumith1896/spoc.git /content/spoc

Cloning into '/content/spoc'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 151 (delta 1), reused 7 (delta 1), pack-reused 143 (from 1)
Receiving objects: 100% (151/151), 20.05 MiB | 22.15 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
# ======================================================
# 🧹 Step 2: Load and Preprocess Dataset (CodeXGLUE)
# ======================================================
from datasets import load_dataset

# Load the Python subset of the CodeXGLUE dataset
dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

# We’ll rename columns to match your project format
df = pd.DataFrame({
    "pseudo": dataset["train"]["docstring"],   # pseudo-code / description
    "code": dataset["train"]["code"]           # target Python code
})

print("✅ Loaded pairs:", df.shape)
print(df.head())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

python/train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

python/train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

python/validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

python/test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

✅ Loaded pairs: (251820, 2)
                                              pseudo  \
0  Set the text for this element.\n\n        Argu...   
1  Associate a document with this element.\n\n   ...   
2  Tests whether a new element of this class can ...   
3  This method will be called after an element is...   
4  Recompute textual value based on the text cont...   

                                                code  
0  def settext(self, text, cls='current'):\n     ...  
1  def setdocument(self, doc):\n        """Associ...  
2  def addable(Class, parent, set=None, raiseexce...  
3  def postappend(self):\n        """This method ...  
4  def updatetext(self):\n        """Recompute te...  


In [4]:
# ======================================================
# 🧩 Step 3: Tokenization & Dataset Formatting
# ======================================================

from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def preprocess(examples):
    """Combine pseudo-code and target code into GPT-2 training format."""
    inputs = [f"<|startoftext|>Pseudo:\n{p}\nCode:\n{c}<|endoftext|>"
              for p, c in zip(examples["pseudo"], examples["code"])]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

from datasets import Dataset

dataset = Dataset.from_pandas(df.sample(20000, random_state=42))  # smaller subset for Colab runtime
split = dataset.train_test_split(test_size=0.1, seed=42)

tokenized = split.map(preprocess, batched=True, remove_columns=["pseudo", "code"])
print("✅ Tokenized dataset:", tokenized)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Tokenized dataset: DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 18000
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})


In [5]:
# ======================================================
# 🧠 Step 4: Fine-Tune GPT-2 (Updated for Transformers ≥4.48)
# ======================================================

from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# Load base GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
model.resize_token_embeddings(len(tokenizer))

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/gpt2-codexglue",         # where to save model
    overwrite_output_dir=True,
    eval_strategy="epoch",                        # ✅ replaced old key (was evaluation_strategy)
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_dir="/content/logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),               # use mixed precision if GPU supports
    report_to=[],                                 # ✅ replaces "report_to='none'" in new versions
    save_total_limit=2,                           # keep last 2 checkpoints only
    learning_rate=5e-5
)

# Data collator (for causal language modeling)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"]
)

# Begin fine-tuning
print("🚀 Starting GPT-2 fine-tuning on pseudo-code to Python dataset...")
trainer.train()

# Save final model
trainer.save_model("/content/gpt2-codexglue/final_model")
tokenizer.save_pretrained("/content/gpt2-codexglue/final_model")

print("✅ Fine-tuning complete. Model saved to /content/gpt2-codexglue/final_model")


✅ Using device: cuda


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

🚀 Starting GPT-2 fine-tuning on pseudo-code to Python dataset...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.157300,1.130083


Epoch,Training Loss,Validation Loss
1,1.157300,1.130083
2,1.053300,1.097526


✅ Fine-tuning complete. Model saved to /content/gpt2-codexglue/final_model


In [ ]:
'''
# ======================================================
# ⚡ Step 4: Fine-Tune GPT-2 (Quick Version for T4 GPU)
# IF YOU WANT TO TRAIN ON SMALL DATASET AND TO TRAIN ON CPU INSTEAD OF GPU
# ======================================================

from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ✅ Load base GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
model.resize_token_embeddings(len(tokenizer))

# ✅ (NEW) Use smaller subset for quick training
# Original: tokenized["train"] & tokenized["test"]
# Reduced to smaller samples for speed
small_train_dataset = tokenized["train"].shuffle(seed=42).select(range(3000))   # was full dataset
small_eval_dataset = tokenized["test"].shuffle(seed=42).select(range(500))      # was full dataset

# ✅ Define training arguments
training_args = TrainingArguments(
    output_dir="/content/gpt2-codexglue",
    overwrite_output_dir=True,
    eval_strategy="epoch",                        # evaluate once per epoch
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,                           # ✅ changed from 2 → 1 epoch
    save_strategy="no",                           # ✅ disabled saving after every epoch
    logging_dir="/content/logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to=[],
    learning_rate=5e-5,
)

# ✅ Data collator for causal LM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ✅ Initialize trainer with smaller dataset
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=small_train_dataset,            # ✅ changed from full dataset
    eval_dataset=small_eval_dataset               # ✅ changed from full dataset
)

# 🚀 Train for 1 epoch on smaller data
print("🚀 Fine-tuning GPT-2 (Quick GPU Version: 1 Epoch, 3k Samples)...")
trainer.train()

# ✅ Save final model
trainer.save_model("/content/gpt2-codexglue/final_model")
tokenizer.save_pretrained("/content/gpt2-codexglue/final_model")

print("✅ Fine-tuning complete. Model saved to /content/gpt2-codexglue/final_model")

'''



In [7]:
# ======================================================
# 📊 Step 5: Evaluation (BLEU) — Fixed Version
# ======================================================

import evaluate
bleu = evaluate.load("sacrebleu")

preds, refs = [], []
sample_eval = df.sample(100, random_state=42)  # evaluate on 100 samples to save time

for row in sample_eval.itertuples():
    prompt = f"Pseudo:\n{row.pseudo}\nCode:\n"
    # ✅ Return both input_ids and attention_mask
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

    # ✅ Use max_new_tokens instead of max_length
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,      # generate up to 200 new tokens
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and clean
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Code:\n")[-1]
    preds.append(pred.strip())
    refs.append([row.code.strip()])

# Compute BLEU score
bleu_score = bleu.compute(predictions=preds, references=refs)
print(f"✅ BLEU Score: {bleu_score['score']:.2f}")


✅ BLEU Score: 20.36


In [11]:
# ======================================================
# 🎨 Step 7: Gradio Interface — GPT-2 Pseudo→Python Generator (Fixed)
# ======================================================

import gradio as gr
import torch

# Load fine-tuned model and tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

model_path = "/content/gpt2-codexglue/final_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
model.eval()

# -------------------------
# Function for code generation
# -------------------------
def generate_code(pseudo):
    if not pseudo.strip():
        return "⚠️ Please enter pseudo-code instructions."

    prompt = f"Pseudo:\n{pseudo}\nCode:\n"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

    # 🚀 Improved generation settings
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,          # balanced output length
        num_beams=3,                 # moderate search
        temperature=0.7,             # creative but stable
        top_p=0.9,                   # nucleus sampling
        repetition_penalty=1.2,      # avoids duplicate lines
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode output and clean it
    code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Code:" in code:
        code = code.split("Code:")[-1]
    return code.strip()

# -------------------------
# 🎨 Gradio Interface
# -------------------------
title = "🤖 GPT-2 — Pseudo-code → Python Code Generator"
description = """
Enter structured pseudo-code instructions (e.g. *"Write a function that checks if a number is prime"*)
and the model will generate the corresponding Python code.
"""

demo = gr.Interface(
    fn=generate_code,
    inputs=gr.Textbox(lines=4, placeholder="Enter your pseudo-code here...", label="Pseudo-code Instructions"),
    outputs=gr.Code(language="python", label="Generated Python Code"),
    title=title,
    description=description,
    examples=[
        ["Write a function that returns the maximum number in a list."],
        ["Define a function that checks if a number is even or odd."],
        ["Write a function that calculates the factorial of a number."],
        ["Create a function that reverses a string."],
    ],
)

# Launch interface
demo.launch(share=True)


✅ Using device: cuda
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06bb5864025577af13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
